![piggy_bank](piggy_bank.jpg)

<br>

Personal loans are a lucrative revenue stream for banks. The typical interest rate of a two year loan in the United Kingdom is [around 10%](https://www.experian.com/blogs/ask-experian/whats-a-good-interest-rate-for-a-personal-loan/). This might not sound like a lot, but in September 2022 alone UK consumers borrowed [around £1.5 billion](https://www.ukfinance.org.uk/system/files/2022-12/Household%20Finance%20Review%202022%20Q3-%20Final.pdf), which would mean approximately £300 million in interest generated by banks over two years!

You have been asked to work with a bank to clean and store the data they collected as part of a recent marketing campaign, which aimed to get customers to take out a personal loan. They plan to conduct more marketing campaigns going forward so would like you to set up a PostgreSQL database to store this campaign's data, designing the schema in a way that would allow data from future campaigns to be easily imported. 

They have supplied you with a csv file called `"bank_marketing.csv"`, which you will need to clean, reformat, and split, in order to save separate files based on the tables you will create. It is recommended to use `pandas` for these tasks.

Lastly, you will write the SQL code that the bank can execute to create the tables and populate with the data from the csv files. As the bank are quite strict about their security, you'll save SQL files as multiline string variables that they can then use to create the database on their end. 

You have been asked to design a database that will have three tables:

## client

| column | data type | description | original column in dataset |
|--------|-----------|-------------|----------------------------|
| `id` | `serial` | Client ID - primary key | `client_id` |
| `age` | `integer` | Client's age in years | `age` |
| `job` | `text` | Client's type of job | `job` |
| `marital` | `text` | Client's marital status | `marital` | 
| `education` | `text` | Client's level of education | `education` |
| `credit_default` | `boolean` | Whether the client's credit is in default | `credit_default` |
| `housing` | `boolean` | Whether the client has an existing housing loan (mortgage) | `housing` | 
| `loan` | `boolean` | Whether the client has an existing personal loan | `loan` |

<br>

## campaign

| column | data type | description | original column in dataset |
|--------|-----------|-------------|----------------------------|
| `campaign_id` | `serial` | Campaign ID - primary key | N/A - new column |
| `client_id` | `serial` | Client ID - references `id` in the `client` table | `client_id` |
| `number_contacts` | `integer` | Number of contact attempts to the client in the current campaign | `campaign` |
| `contact_duration` | `integer` | Last contact duration in seconds | `duration` |
| `pdays` | `integer` | Number of days since contact in previous campaign (`999` = not previously contacted) | `pdays` |
| `previous_campaign_contacts` | `integer` | Number of contact attempts to the client in the previous campaign | `previous` |
| `previous_outcome` | `boolean` | Outcome of the previous campaign | `poutcome` |
| `campaign_outcome` | `boolean` | Outcome of the current campaign | `y` |
| `last_contact_date` | `date` | Last date the client was contacted | A combination of `day`, `month`, and the newly created `year` |

<br>

## economics

| column | data type | description | original column in dataset |
|--------|-----------|-------------|----------------------------|
| `client_id` | `serial` | Client ID - references `id` in the `client` table | `client_id` |
| `emp_var_rate` | `float` | Employment variation rate (quarterly indicator) | `emp_var_rate` |
| `cons_price_idx` | `float` | Consumer price index (monthly indicator) | `cons_price_idx` |
| `euribor_three_months` | `float` | Euro Interbank Offered Rate (euribor) three month rate (daily indicator) | `euribor3m` |
| `number_employed` | `float` | Number of employees (quarterly indicator)| `nr_employed` |

In [239]:
import pandas as pd

import numpy as np

import sqlite3

df = pd.read_csv("bank_marketing.csv")

df.head()

# Add 'campaign_id' and year' to campaign and assign values to them
campaign = df.assign(campaign_id = 1, year = '2022')

campaign = campaign.rename(columns = {'duration':'contact_duration','previous':'previous_campaign_contacts', 'poutcome':'previous_outcome','y':'campaign_outcome','campaign':'number_contacts'})

campaign['concatenated_date'] = campaign['year'] + '-' + campaign['month'].str.zfill(2) + '-' + campaign['day'].astype(str).str.zfill(2)

# Convert the 'concatenated_date' column to datetime format
campaign['last_contact_date'] = pd.to_datetime(campaign['concatenated_date'])

# Drop the uncessary columns from campaign data frame
campaign.drop(columns=['age','job', 'contact', 'year', 'marital', 'education', 'month', 'credit_default', 'housing', 'loan', 'day', 'emp_var_rate', 'cons_price_idx', 'cons_conf_idx', 'euribor3m', 'nr_employed', 'concatenated_date'], inplace=True)

# Reorder the columns in campaign
campaign = campaign[['campaign_id','client_id','number_contacts','contact_duration','pdays','previous_campaign_contacts','previous_outcome','campaign_outcome','last_contact_date']]

campaign.head()

# Subset df based on a range of columns and name the data frame 'client'
client = df.loc[:, 'client_id':'loan']

client = client.rename(columns = {'client_id':'id'})

client.head()

economics = df.rename(columns = {'euribor3m':'euribor_three_months', 'nr_employed':'number_employed'})

economics = economics[['client_id','emp_var_rate','cons_price_idx','euribor_three_months','number_employed']]

economics.head()

,client_id,emp_var_rate,cons_price_idx,euribor_three_months,number_employed
0,0,1.1,93.994,4.857,5191.0
1,1,1.1,93.994,4.857,5191.0
2,2,1.1,93.994,4.857,5191.0
3,3,1.1,93.994,4.857,5191.0
4,4,1.1,93.994,4.857,5191.0


In [240]:
client.head()

,id,age,job,marital,education,credit_default,housing,loan
0,0,56,housemaid,married,basic.4y,no,no,no
1,1,57,services,married,high.school,unknown,no,no
2,2,37,services,married,high.school,no,yes,no
3,3,40,admin.,married,basic.6y,no,no,no
4,4,56,services,married,high.school,no,no,yes


In [241]:
campaign.head()

,campaign_id,client_id,number_contacts,contact_duration,pdays,previous_campaign_contacts,previous_outcome,campaign_outcome,last_contact_date
0,1,0,1,261,999,0,nonexistent,no,2022-05-13
1,1,1,1,149,999,0,nonexistent,no,2022-05-19
2,1,2,1,226,999,0,nonexistent,no,2022-05-23
3,1,3,1,151,999,0,nonexistent,no,2022-05-27
4,1,4,1,307,999,0,nonexistent,no,2022-05-03


In [242]:
# Clean the "education" column, changing "." to "_" and "unknown" to NumPy's null values

metacharacter_to_match = r'\.'  # For example, matching '.'
replacement_metacharacter = '_'  # Replace '.' with '*'

# Use regex to replace the metacharacter in the DataFrame
client['education'].replace(to_replace=metacharacter_to_match, value=replacement_metacharacter, regex=True, inplace=True)

string_to_replace = 'unknown'
# Use replace to replace 'unknown' with NumPy null values
client.replace(to_replace=string_to_replace, value=np.nan, inplace=True)

client.head()

,id,age,job,marital,education,credit_default,housing,loan
0,0,56,housemaid,married,basic_4y,no,no,no
1,1,57,services,married,high_school,NaN,no,no
2,2,37,services,married,high_school,no,yes,no
3,3,40,admin.,married,basic_6y,no,no,no
4,4,56,services,married,high_school,no,no,yes


In [243]:
# Remove periods from the "job" column
client['job'].replace(to_replace = metacharacter_to_match, value = '', regex = True, inplace = True)

client.head()

,id,age,job,marital,education,credit_default,housing,loan
0,0,56,housemaid,married,basic_4y,no,no,no
1,1,57,services,married,high_school,NaN,no,no
2,2,37,services,married,high_school,no,yes,no
3,3,40,admin,married,basic_6y,no,no,no
4,4,56,services,married,high_school,no,no,yes


In [244]:
# Convert "success" and "failure" in the "previous_outcome" and change "nonexistent" to NumPy's null values.

strings_to_match = ['success', 'failure', 'nonexistent']
values_to_replace = [1, 0, np.nan]

campaign['previous_outcome'].replace(to_replace=strings_to_match, value=values_to_replace, inplace=True)

campaign['previous_outcome']

# Convert "campaign_outcome" columns to binary (1 or 0).

strings_to_match2 = ['yes', 'no']
values_to_replace2 = [1, 0]

campaign['campaign_outcome'].replace(to_replace=strings_to_match2, value=values_to_replace2, inplace=True)

campaign['campaign_outcome']

0        0
1        0
2        0
3        0
4        0
        ..
41183    1
41184    0
41185    0
41186    1
41187    0
Name: campaign_outcome, Length: 41188, dtype: int64

In [245]:
campaign.tail()


,campaign_id,client_id,number_contacts,contact_duration,pdays,previous_campaign_contacts,previous_outcome,campaign_outcome,last_contact_date
41183,1,41183,1,334,999,0,NaN,1,2022-11-30
41184,1,41184,1,383,999,0,NaN,0,2022-11-06
41185,1,41185,2,189,999,0,NaN,0,2022-11-24
41186,1,41186,1,442,999,0,NaN,1,2022-11-17
41187,1,41187,3,239,999,1,0.0,0,2022-11-23


In [246]:
client.dtypes


id                 int64
age                int64
job               object
marital           object
education         object
credit_default    object
housing           object
loan              object
dtype: object

In [247]:
# Save the DataFrames as CSV files
client.to_csv(index=False)
campaign.to_csv(index=False)
economics.to_csv(index=False)

'client_id,emp_var_rate,cons_price_idx,euribor_three_months,number_employed\n0,1.1,93.994,4.857,5191.0\n1,1.1,93.994,4.857,5191.0\n2,1.1,93.994,4.857,5191.0\n3,1.1,93.994,4.857,5191.0\n4,1.1,93.994,4.857,5191.0\n5,1.1,93.994,4.857,5191.0\n6,1.1,93.994,4.857,5191.0\n7,1.1,93.994,4.857,5191.0\n8,1.1,93.994,4.857,5191.0\n9,1.1,93.994,4.857,5191.0\n10,1.1,93.994,4.857,5191.0\n11,1.1,93.994,4.857,5191.0\n12,1.1,93.994,4.857,5191.0\n13,1.1,93.994,4.857,5191.0\n14,1.1,93.994,4.857,5191.0\n15,1.1,93.994,4.857,5191.0\n16,1.1,93.994,4.857,5191.0\n17,1.1,93.994,4.857,5191.0\n18,1.1,93.994,4.857,5191.0\n19,1.1,93.994,4.857,5191.0\n20,1.1,93.994,4.857,5191.0\n21,1.1,93.994,4.857,5191.0\n22,1.1,93.994,4.857,5191.0\n23,1.1,93.994,4.857,5191.0\n24,1.1,93.994,4.857,5191.0\n25,1.1,93.994,4.857,5191.0\n26,1.1,93.994,4.857,5191.0\n27,1.1,93.994,4.857,5191.0\n28,1.1,93.994,4.857,5191.0\n29,1.1,93.994,4.857,5191.0\n30,1.1,93.994,4.857,5191.0\n31,1.1,93.994,4.857,5191.0\n32,1.1,93.994,4.857,5191.0\n33,1.1,93

In [248]:
client_table = """
CREATE TABLE client (
	id SERIAL PRIMARY KEY,
	age INT,
	job TEXT,
	marital TEXT,
	education TEXT,
	credit_default BOOL,
	housing BOOL,
	loan BOOL
);

\COPY client FROM 'client.csv' DELIMITER ',' CSV HEADER;

"""

In [249]:
client_table = """
CREATE TABLE client (
	id SERIAL PRIMARY KEY,
	age INT,
	job TEXT,
	marital TEXT,
	education TEXT,
	credit_default BOOL,
	housing BOOL,
	loan BOOL
);

\COPY client FROM 'client.csv' DELIMITER ',' CSV HEADER;

"""

In [250]:
campaign_table = """
CREATE TABLE campaign (
    campaign_id SERIAL PRIMARY KEY,
    client_id SERIAL references client (id)
    number_contacts INT,
    contact_duration INT,
    pdays INT,
    previous_campaign_contacts INT,
    previous_outcome BOOL,
    campaign_outcome BOOL,
    last_contact_date DATE
);

\COPY campaign FROM 'campaign.csv' DELIMITER ',' CSV HEADER;
"""


In [251]:
economics_table = """
CREATE TABLE economics (
	client_id SERIAL references client (id),
	emp_var_rate FLOAT,
	cons_price_idx FLOAT,
	euribor_three_months FLOAT,
	number_employed FLOAT
);

\COPY economics FROM 'economics.csv' DELIMITER ',' CSV HEADER;
"""